# User Testing Planning/Coding Notebook

### Objectives:
- Plan user testing
- Code scripts to generate test maps:
    - Randomly pull maps/music from 30 songs with maps for all difficulty levels from Beat Saver (tried all >70% rating but not enough were rated)
    - Strip obstacles/lighting events from human made maps and set background environment all the same
    - Map all songs with each model type
    - Automatically categorize songs/beat structure?
- For each user/volunteer:
    - Randomly assign Random, HMM, Segmented HMM, Rate-Seg HMM, or Human-made
    - Record email address, user-ID, and assigned map information (song, model/human, difficulty) in pd DataFrame
    - Send assignment to user    

In [2]:
import pandas as pd
import numpy as np
import json
import requests
import zipfile
import pickle

In [5]:
with open("../../data/metadata.pkl", 'rb') as file:
    metadata = pickle.load(file)

In [6]:
metadata[0]

{'metadata': {'difficulties': {'easy': False,
   'expert': False,
   'expertPlus': True,
   'hard': False,
   'normal': False},
  'duration': 268,
  'characteristics': [{'difficulties': {'easy': None,
     'expert': None,
     'expertPlus': {'duration': 771.25,
      'length': 265,
      'njs': 20,
      'njsOffset': 0,
      'bombs': 30,
      'notes': 1986,
      'obstacles': 3},
     'hard': None,
     'normal': None},
    'name': 'Standard'}],
  'levelAuthorName': 'Durple',
  'songAuthorName': 'RIOT',
  'songName': 'Knife Party & Tom Morello - Battle Sirens (RIOT Remix)',
  'songSubName': '',
  'bpm': 174},
 'stats': {'downloads': 107,
  'plays': 0,
  'downVotes': 0,
  'upVotes': 0,
  'heat': 1186.1311706,
  'rating': 0},
 'description': 'Fully Tested and Re-Vamped \nsome parts are challenging :) ',
 'deletedAt': None,
 '_id': '5e16219eb9d8a90006ede70f',
 'key': '7d17',
 'name': 'Knife Party & Tom Morello - Battle Sirens (RIOT Remix)',
 'uploader': {'_id': '5dca50351a3ccd000666bbce

In [7]:
metadata[0]['stats']

{'downloads': 107,
 'plays': 0,
 'downVotes': 0,
 'upVotes': 0,
 'heat': 1186.1311706,
 'rating': 0}

In [18]:
potential_songs = []

for x in metadata:
    if all(value == True for value in x['metadata']['difficulties'].values()):
        potential_songs.append(x)

In [19]:
len(potential_songs)

556

In [20]:
test_songs = np.random.choice(potential_songs, 30, replace = False)

For each test song in the list:
1. Download with key and unzip
2. For each difficulty level:
    - Map with each model
    - Edit/strip human map
    - Save zip folder

In [26]:
keys = []
for x in test_songs:
    keys.append(x['key'])

In [28]:
def open_map_file(difficulty):
    """This function opens the map file listed in the info.dat file for the specificed difficulty level."""
    with open('./temp/info.dat', 'rb') as i:
        info = json.load(i)
    for x in info['_difficultyBeatmapSets']:
        if x['_beatmapCharacteristicName'] == 'Standard':
            for y in x['_difficultyBeatmaps']:
                if y['_difficulty'] == difficulty:
                    file_name = y['_beatmapFilename']
                    with open(f"./temp/{file_name}", 'rb') as f:
                        map_file = json.load(f)
                
    return map_file

In [29]:
def download_song_and_map(key):
    """Downloads the zipped folder of song and mapping data from the beatsaber api. Extracts files to a 'temp' folder 
    in the local directory."""
    response = requests.get(f"https://beatsaver.com/api/download/key/{key}")
    if response.status_code == 200:
        content_as_file = BytesIO(response.content)
        zip_file = ZipFile(content_as_file)
        for x in zip_file.filelist:
            print(zip_file.extract(x.filename, path = 'temp'))
        return response.status_code
    else:
        return print(f"API call failed at {key} with error code {response.status_code}")